## <div style="padding: 50px;color:white;margin:10;font-size:80%;text-align:left;display:fill;border-radius:10px;background-color:#323232;overflow:hidden"><b><span style='color:#F1A424'>(15/16) |</span></b> Кластеризация Текстов</div>

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#F1A424'>НАША ЗАДАЧА</span></b></p></div>

Рассмотрим пример кластеризации текстов на стандартном наборе данных, новостях, выберем в наборе **4 из 20 категорий**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#F1A424'>СКАЧИВАЕМ ДАННЫЕ</span></b></p></div>

Используя `fetch_20newsgroups`

In [ ]:
# Выбираем 4 категории новостей для легковесности примера
categories = [
    'rec.sport.hockey', # хоккей
    'talk.politics.mideast', # политические новости о Ближнем Востоке
    'comp.graphics', # компьютерная графика
    'sci.crypt' # криптография
]

# Скачиваем набор данных
dataset = fetch_20newsgroups(subset='all', categories=categories,
                             shuffle=True, random_state=42)

print("%d документов" % len(dataset.data))
print("%d категории" % len(dataset.target_names))

# Записываем значения категорий для каждой новости
labels = dataset.target
labels[:10]

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#F1A424'>ТОКЕНИЗАЦИЯ</span></b></p></div>

Тексты новостей нужно преобразовать в числа и почистить от посторонних символов. Для этого воспользуемся инструментом из библиотеки sklearn, он построит `токенизированные` данные.

> Примечание: Токенизация — это разделение текста на компоненты. Например, токенизация по предложениям — это процесс разделения письменного языка на предложения-компоненты. А токенизация по словам — это разделение предложений на слова-компоненты.

In [ ]:
# Создаём объект, который будет токенизировать данные
analyzer = CountVectorizer(stop_words='english').build_analyzer()

# Токенизируем набор данных
docs = []
for document in dataset.data:
    docs.append(analyzer(document.replace('_', '')))

# Первые 10 слов первого документа
docs[0][:10]

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#F1A424'>ВЕКТОРИЗАЦИЯ СЛОВ</span></b></p></div>

Каждый текст превратился в отдельный набор слов. Перейдём к векторизации текстов: воспользуемся моделью Word2Vec, которая для каждого слова строит числовой вектор, в нашем случае каждое слово будет кодироваться 50 числами. При этом вектора будем делать для тех слов, которые встречаются больше 20 раз во всех текстах документов. Также сделаем средний вектор для всех слов, таким образом получим вектор для текста в целом.

In [ ]:
from gensim.models import Word2Vec

# Обучаем модель векторайзера на нашем наборе данных
# На выходе мы получим вектор признаков для каждого слова
model = Word2Vec(docs, min_count=20, vector_size=50)

# Наивный подход к созданию единого эмбеддинга для документа – средний эмбеддинг по словам
def doc_vectorizer(doc, model):
    doc_vector = []
    num_words = 0
    for word in doc:
        try:
            if num_words == 0:
                doc_vector = model.wv[word]
            else:
                doc_vector = np.add(doc_vector, model.wv[word])
            num_words += 1
        except:
            # pass if word is not found
            pass
     
    return np.asarray(doc_vector) / num_words

X = []
for doc in tokenized_corpus:
    X.append(doc_vectorizer(doc, w2v_model))
    
print(f'Sentences: {len(X)}')
print(f'Each sentence has {X[0].shape} dimensions')

In [ ]:
X[0]

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#F1A424'>УМЕНЬШЕНИЯ РАЗМЕРНОСТИ</span></b></p></div>

Наша кластеризация будет работать лучше, если снизить размерность, для этого воспользуемся методом `TSNE`

> Примечание: `TSNE` — это техника нелинейного снижения размерности и визуализации многомерных признаков, почитать о ней можно, например, здесь.

При понижении размерности мы сохраняем близость элементов, то есть если элементы были близки при `vector_size=50`, то они останутся близки и при `vector_size=2`



In [ ]:
# t-SNE – метод понижения размерности
from sklearn.manifold import TSNE

# Создаём объект для выполнения t-SNE
tsne = TSNE(n_components=2, random_state=0)

# Преобразуем наши данные, понизив размерность с 50 до 2
X = tsne.fit_transform(X)

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#F1A424'>КЛАСТЕРИЗАЦИЯ ДЛЯ ВИЗУАЛИЗАЦИИ</span></b></p></div>

Создадим кластеризатор `KMeans` и обучим на подготовленных данных. Мы можем посмотреть получившиеся центроиды и предсказанные кластеры.

In [ ]:
# Создаём KMeans кластеризатор 
kmeans = KMeans(n_clusters=4)

# Обучаем кластеризатор на подготовленных данных
kmeans.fit(X)

# Получаем предсказанные кластеры
y_pred = kmeans.labels_.astype(np.int)

# Координаты полученных центроидов
print ("Координаты центроидов:\n", kmeans.cluster_centers_)